In [ ]:
from pstatsviewer.viewer import _calc_frames
from pstats import Stats
import pandas as pd

In [ ]:
def enrich_stas_df(df):
    return pd.merge(
        (
            df.reset_index()["filename:funcname"]
                .str.split(":", n = 1, expand = True)
                .rename(columns={0:"file", 1: "function"})
                .assign(**{
                    "filename:funcname": df.reset_index()["filename:funcname"],
                    "module": [ 
                        row[-1:][0].split(":")[0].replace(".py", "")
                        for row in df.reset_index()["filename:funcname"].str.split("/")
                    ]
                })
            
        ),
        df.reset_index(),
        on="filename:funcname"
    ).drop(['filename:funcname'], axis=1)


In [ ]:
generator_timings_df = enrich_stas_df(_calc_frames(Stats('../metrics/test_generator_consumption.prof'))[0])
iterator_timings_df = enrich_stas_df(_calc_frames(Stats('../metrics/test_iterator_consumption.prof'))[0])
chain_gen_timings_df = enrich_stas_df(_calc_frames(Stats('../metrics/test_chained_generator_consumption.prof'))[0])
chain_iter_timings_df = enrich_stas_df(_calc_frames(Stats('../metrics/test_chained_iterator_consumption.prof'))[0])

df = pd.concat([
    generator_timings_df.assign(type="generator"), 
    iterator_timings_df.assign(type="iterator"), 
    chain_gen_timings_df.assign(type="chained-generator"),
    chain_iter_timings_df.assign(type="chained-iterator")
]).sort_values(by=["function", "cumtime"])

In [ ]:
df[df.function.str.startswith("test_consuming")].head(20)

In [ ]:
df[(
    (df.type == "iterator") & 
    df.function.str.startswith("test_consuming")
)].sort_values(by=["module", "cumtime"])